## baseline

In [19]:
import nltk
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/manuelcostareis/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [20]:
import pandas as pd
import numpy as np

import re
import string
from numpy import inf

# Baseline 
from collections import Counter, OrderedDict
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import RSLPStemmer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
stop_words = stopwords.words('portuguese')
import warnings; warnings.simplefilter('ignore')


# SKLearn related imports
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


# Phrase modelling
from gensim.models.phrases import Phrases, Phraser


random_state = 42

In [22]:
df = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [23]:
df.head()

,Request,Label
0,Adicionar música tomohisa yamashita à minha li...,add_to_playlist
1,Eu quero adicionando Aprite le finestre à minh...,add_to_playlist
2,Que curta-metragens estão tocando às 11 da man...,search_screening_event
3,precisa de uma sala em um que serve foie gras ...,book_restaurant
4,"Atribuir 4 estrelas de 6 para a crônica, Deus ...",rate_book


In [24]:
df.describe()

,Request,Label
count,26527,26799
unique,25752,8
top,Encontre o horário do filme.,no_intent
freq,5,12645


In [25]:
df.Request.iloc[10]

'Toca a melhor música de Phoebe Snow'

In [26]:
df.Label.unique()

array(['add_to_playlist', 'search_screening_event', 'book_restaurant',
       'rate_book', 'get_weather', 'play_music', 'search_creative_work',
       'no_intent'], dtype=object)

## Train Test Split

In [27]:
df = df.dropna()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    df.Request, df.Label, test_size=0.33, random_state=random_state)

In [29]:
tokenizer = WordPunctTokenizer()
stemmer = RSLPStemmer()
regex_list = [("<[^>]*>", "")]

## Tokenize and Stem

In [33]:
phrases = Phrases(X_train, min_count=1, threshold=1)

In [37]:
bigram = Phraser(phrases)

In [42]:
bigram[X_train]

['Vai ser parte do mistério o tempo todo, porque se nasceu em Valparaíso e foi inscrito em Santiago, vai constar na certidão como Santiago.',
 'É claro que ele deu, porque ele preferiria apodrecer as células do cérebro do que passar tempo de qualidade com você.',
 'Adicione uma faixa de camui gackt à lista de reprodução do White Noise.',
 'Posso reservas colocar a chaleira no fogo para o chá?',
 'Encontre Phalcon.',
 'Adicionar karusellen ao jazz brasileiro',
 'Quero olhar para os tempos de escola e ver que comecei devagar, mas acabei bem.',
 'Vai ser bom como no pouco de Versalhes.',
 'É em algum lugar novo no centro da cidade -- -',
 'Tocar concerto de música de Nicholaus Arson.',
 'Avalie o atual cinco de seis luas',
 'Você pode me encontrar o jogo, rei do Raop?',
 'Encontre o carro, encontre o assassino.',
 'É um sacrilégio para o homem falar os segredos de Hera.',
 'Vamos meninas, vamos ombrear o Sr Maddox.',
 'Posso colocar o johnny burke na minha lista de reprodução?',
 'Vamos b

In [30]:
# Custom transformer to implement sentence cleaning
class PortugueseCleaner(TransformerMixin):
    def __init__(self, tokenizer, stemmer, phrases, regex_list,
                 lower=True, remove_punct=True):
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.regex_list = regex_list
        self.phrases = phrases
        self.lower = lower
        self.remove_punct = remove_punct
        
    def transform(self, X, *_):
        X = list(map(self._clean_sentence, X))
        return X
    
    def _clean_sentence(self, sentence):
        
        # Replace given regexes
        #for regex in self.regex_list:
        #    sentence = re.sub(regex[0], regex[1], sentence)
            
        # lowercase
        if self.lower:
            sentence = Phraser()

        # Split sentence into list of words
        words = self.tokenizer.tokenize(sentence)
        
        if self.phrases:
            phrases = Phrases(sentence, min_count=1, threshold=1)
            
        # Remove punctuation
        if self.remove_punct:
            words = list(filter(lambda x: x not in string.punctuation, words))

        # Stem words
        if self.stemmer:
            words = map(self.stemmer.stem, words)

        # Join list elements into string
        sentence = " ".join(words)
        
        return sentence
    
    def fit(self, *_):
        return self

## Pipeline

In [31]:
def pipeliner():
    k = 5000
    tokenizer = WordPunctTokenizer()
    stemmer = RSLPStemmer()
    phrases = Phrases(sentences, min_count=1, threshold=1)
    regex_list = [("<[^>]*>", "")]
    
    return Pipeline([('stemm', PortugueseCleaner(tokenizer, stemmer, phrases=phrases, regex_list, lower=False)),
                     ('vect', TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words, max_features=20000)),
                     ('selectKBest', SelectKBest(chi2, k=k)),
                     ('classifier', RandomForestClassifier(n_jobs=-1, n_estimators=2000, class_weight='balanced_subsample'))])

In [32]:
# Build the pipeline

if 1 == 1:
    pipe = pipeliner()
    # Train the classifier
    pipe.fit(X_train, y_train)

    predicted = pipe.predict(X_test)
    mean = np.mean(predicted == y_test)


NameError: name 'sentences' is not defined

In [ ]:
mean